# Imports

In [1]:
from imutils.video import FileVideoStream, FPS
import sys
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')    
import os
import cv2
import time
import argparse
import multiprocessing
import numpy as np
import tensorflow as tf
import threading

sys.path.append('..')
sys.path.append('../utils/')

from multiprocessing import Queue, Pool
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Env setup

In [2]:


PATH_TO_MODEL = "../models/ssd_mobilenet_v1_coco/frozen_inference_graph.pb"
PATH_TO_LABELS = "../object_detection/data/mscoco_label_map.pbtxt"
PATH_TO_VIDEO = "../data/test_video.mp4"

NUM_CLASSES = 90

# Load label map

In [3]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES,
                                                           use_display_name = True)
category_index = label_map_util.create_category_index(categories)

# Allow memory growth

In [4]:
# Session Config: Limit GPU Memory Usage
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

# Load frozen inference graph

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

# Run detection

In [7]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph, config = config) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Bounding boxes
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Start Videofile stream
        print("[INFO] starting video stream")
        video_stream = FileVideoStream(PATH_TO_VIDEO).start()
        # let queue fill up
        time.sleep(3)
        # Warmup on a dummy image
        print("[INFO] warming up on dummy image")
        image_np_expanded = 128 * np.ones((1,300, 300, 3), dtype=np.uint8)
        for i in range(2):
            (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})

        # fps calculation
        fps = FPS().start()
        frame_cnt = 0
        detect_accumulated = 0
        print("[INFO] starting inference")
        try:
            while video_stream.more():
                # read frame from video
                image_np = video_stream.read()
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                # Actual detection.
                t_detect = time.time()
                (boxes, scores, classes, num) = sess.run(
                    [detection_boxes, detection_scores, detection_classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                detect_accumulated = detect_accumulated + (time.time() - t_detect)

                #update framcounter
                frame_cnt = frame_cnt + 1
                if (frame_cnt % 50)== 0:
                    print("[INFO] detection FPS: {:.2f}".format(frame_cnt/detect_accumulated))
                    #print("[INFO] total FPS: {:.2f}".format(fps.fps()))
                    print("[INFO] Frame count: {:d}".format(frame_cnt))
                    
                # fps calculation
                fps.update()                
        except KeyboardInterrupt:
            pass


        # End everything
        print("[INFO] Reached end of file - stopping inference")
        fps.stop()
        video_stream.stop()     

[INFO] starting video stream
[INFO] warming up on dummy image
[INFO] starting inference
[INFO] detection FPS: 4.59
[INFO] Frame count: 50
[INFO] detection FPS: 4.61
[INFO] Frame count: 100
[INFO] detection FPS: 4.62
[INFO] Frame count: 150
[INFO] detection FPS: 4.63
[INFO] Frame count: 200
[INFO] detection FPS: 4.63
[INFO] Frame count: 250
[INFO] detection FPS: 4.62
[INFO] Frame count: 300
[INFO] detection FPS: 4.60
[INFO] Frame count: 350
[INFO] detection FPS: 4.60
[INFO] Frame count: 400
[INFO] detection FPS: 4.61
[INFO] Frame count: 450
[INFO] detection FPS: 4.62
[INFO] Frame count: 500
[INFO] detection FPS: 4.62
[INFO] Frame count: 550
[INFO] detection FPS: 4.63
[INFO] Frame count: 600
[INFO] detection FPS: 4.63
[INFO] Frame count: 650
[INFO] detection FPS: 4.64
[INFO] Frame count: 700
[INFO] detection FPS: 4.64
[INFO] Frame count: 750
[INFO] detection FPS: 4.65
[INFO] Frame count: 800
[INFO] detection FPS: 4.66
[INFO] Frame count: 850
[INFO] detection FPS: 4.68
[INFO] Frame count:

# Summary

In [8]:
print('----------------------------------------------------')
print('Summary:')
print('FPS for detection: {:.2f}'.format(frame_cnt/detect_accumulated))
print('FPS entire processing: {:.2f}'.format(fps.fps()))
print('Time taken to post-process 31s videofile: {:.2f}s'.format(fps.elapsed()))

----------------------------------------------------
Summary:
FPS for detection: 4.68
FPS entire processing: 4.67
Time taken to post-process 31s videofile: 194.06s
